In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

#since I may get warnings, I can do the following
import warnings
warnings.filterwarnings("ignore")

# Import dataset

In [2]:
#to load csv
df = pd.read_csv('ch3_dataset.csv')
len(df)

7043

 # Beginning examination of the data

In [3]:
#look at first rows
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
#look at column names and change all uppercase to lowercase
df.columns = df.columns.str.lower().str.replace(' ', '_')
df.columns

Index(['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'tenure', 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod', 'monthlycharges', 'totalcharges', 'churn'],
      dtype='object')

In [5]:
#to look at the datatypes
df.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges         object
churn                object
dtype: object

In [6]:
#from above, seniorcitizen should not be an integer; it should be an object
# also from above, totalcharges should not be an object; it should be a float64
#I tried to do this before but didn't know how to convert the data

# Converting data types

In [7]:
#to convert totalcharges from object to numeric and fill with a 0
df['totalcharges'] = pd.to_numeric(df['totalcharges'], errors='coerce')
df['totalcharges'] = df['totalcharges'].fillna(0)

In [8]:
#This changes data and not just column names to lowercase
df.columns = df.columns.str.lower().str.replace(' ', '_')

string_columns = list(df.dtypes[df.dtypes == 'object'].index)

for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')

In [9]:
#we want 0's and 1's instead of yes and nos for churn
#to change all the yeses to int
#GAH!!!! I totally needed this before
df.churn = (df.churn == 'yes').astype(int)

In [10]:
#transposes
df.head().T

,0,1,2,3,4
customerid,7590-vhveg,5575-gnvde,3668-qpybk,7795-cfocw,9237-hqitu
gender,female,male,male,male,female
seniorcitizen,0,0,0,0,0
partner,yes,no,no,no,no
dependents,no,no,no,no,no
tenure,1,34,2,45,2
phoneservice,no,yes,yes,no,yes
multiplelines,no_phone_service,no,no,no_phone_service,no
internetservice,dsl,dsl,dsl,dsl,fiber_optic
onlinesecurity,no,yes,yes,yes,no


# Set up validation framework

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
#split data into two parts (not three)
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [13]:
#now split up the full training into training and validation df's
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=11)

In [14]:
#to see sizes of the three datasets
len(df_train), len(df_val), len(df_test)

(4225, 1409, 1409)

In [15]:
#reset the indexes
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [16]:
#define your y values
y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

#delete the churn columns
del df_train['churn']
del df_val['churn']
del df_test['churn']

# Exploratory Data Analysis

In [18]:
df_train_full

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
1814,5442-pptjy,male,0,yes,yes,12,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,19.70,258.35,0
5946,6261-rcvns,female,0,no,no,42,yes,no,dsl,yes,...,yes,yes,no,yes,one_year,no,credit_card_(automatic),73.90,3160.55,1
3881,2176-osjuv,male,0,yes,no,71,yes,yes,dsl,yes,...,no,yes,no,no,two_year,no,bank_transfer_(automatic),65.15,4681.75,0
2389,6161-erdgd,male,0,yes,yes,71,yes,yes,dsl,yes,...,yes,yes,yes,yes,one_year,no,electronic_check,85.45,6300.85,0
3676,2364-ufrom,male,0,no,no,30,yes,no,dsl,yes,...,no,yes,yes,no,one_year,no,electronic_check,70.40,2044.75,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905,0781-lkxbr,male,1,no,no,9,yes,yes,fiber_optic,no,...,yes,no,yes,yes,month-to-month,yes,electronic_check,100.50,918.60,1
5192,3507-gasnp,male,0,no,yes,60,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,19.95,1189.90,0
3980,8868-wozgu,male,0,no,no,28,yes,yes,fiber_optic,no,...,yes,no,yes,yes,month-to-month,yes,electronic_check,105.70,2979.50,1
235,1251-krreg,male,0,no,no,2,yes,yes,dsl,no,...,no,no,no,no,month-to-month,yes,mailed_check,54.40,114.10,1


In [19]:
#to reset the index
df_train_full = df_train_full.reset_index(drop=True)
df_train_full

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,5442-pptjy,male,0,yes,yes,12,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,19.70,258.35,0
1,6261-rcvns,female,0,no,no,42,yes,no,dsl,yes,...,yes,yes,no,yes,one_year,no,credit_card_(automatic),73.90,3160.55,1
2,2176-osjuv,male,0,yes,no,71,yes,yes,dsl,yes,...,no,yes,no,no,two_year,no,bank_transfer_(automatic),65.15,4681.75,0
3,6161-erdgd,male,0,yes,yes,71,yes,yes,dsl,yes,...,yes,yes,yes,yes,one_year,no,electronic_check,85.45,6300.85,0
4,2364-ufrom,male,0,no,no,30,yes,no,dsl,yes,...,no,yes,yes,no,one_year,no,electronic_check,70.40,2044.75,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,0781-lkxbr,male,1,no,no,9,yes,yes,fiber_optic,no,...,yes,no,yes,yes,month-to-month,yes,electronic_check,100.50,918.60,1
5630,3507-gasnp,male,0,no,yes,60,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,19.95,1189.90,0
5631,8868-wozgu,male,0,no,no,28,yes,yes,fiber_optic,no,...,yes,no,yes,yes,month-to-month,yes,electronic_check,105.70,2979.50,1
5632,1251-krreg,male,0,no,no,2,yes,yes,dsl,no,...,no,no,no,no,month-to-month,yes,mailed_check,54.40,114.10,1


In [20]:
#to look for missing values
df_train_full.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [21]:
#we good
#so look at the target (dependent) variable churn
df_train_full.churn.value_counts()

0    4113
1    1521
Name: churn, dtype: int64

In [22]:
#1 is yes churn, 0 is no churn
#to normalize and get percentage
df_train_full.churn.value_counts(normalize=True)

0    0.730032
1    0.269968
Name: churn, dtype: float64

In [23]:
#So 26.99% of users are leaving/churning. Global churn rate = 26.99%

In [24]:
#now we want to look at categorical variables to see how many unique values are in each variable
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
               'phoneservice', 'multiplelines', 'internetservice',
               'onlinesecurity', 'onlinebackup', 'deviceprotection',
               'techsupport', 'streamingtv', 'streamingmovies',
               'contract', 'paperlessbilling', 'paymentmethod']
#select a subset 
df_train_full[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

In [25]:
#we can also set up a list for numerical values
numerical = ['tenure', 'monthlycharges', 'totalcharges']

# Churn rate

In [29]:
#global churn
global_churn = df_train_full.churn.mean()
global_churn

0.26996805111821087

In [27]:
#churn rate within each group of one categorical variable  gender
female_mean = df_train_full[df_train_full.gender == 'female'].churn.mean()
print('gender == female:', round(female_mean, 3))

male_mean = df_train_full[df_train_full.gender == 'male'].churn.mean()
print('gender == male:  ', round(male_mean, 3))

gender == female: 0.277
gender == male:   0.263


In [28]:
#churn rate within partner variable

partner_yes = df_train_full[df_train_full.partner == 'yes'].churn.mean()
print('partner == yes:', round(partner_yes, 3))

partner_no = df_train_full[df_train_full.partner == 'no'].churn.mean()
print('partner == no :', round(partner_no, 3))

partner == yes: 0.205
partner == no : 0.33


In [31]:
#take the global mean and subtract from it each submean
global_churn - female_mean

-0.006855983216553063

In [32]:
global_churn - male_mean

0.006754520462819769

In [33]:
global_churn - partner_yes

0.06493474245795922

In [34]:
global_churn - partner_no

-0.05984095297455855

In [35]:
#So gender doesn't really matter for predicting churn rate because differences are small number 

#But having a partner or not may matter
#and since having a partner (yes) is positive, they are less likely to churn than the rest of the group
#not having a partner (no) is negative, so they are more likely to churn. Not having a partner is more likely to churn

# Risk Ratio

In [36]:
from IPython.display import display

In [38]:
#to define the global mean
global_mean = df_train_full.churn.mean()
global_mean

0.26996805111821087

In [39]:
#to look at risk ratio for one variable partner
df_group = df_train_full.groupby(by='partner').churn.agg(['mean'])
df_group['diff'] = df_group['mean'] - global_mean
df_group['risk'] = df_group['mean'] / global_mean
df_group

,mean,diff,risk
partner,,,
no,0.329809,0.059841,1.221659
yes,0.205033,-0.064935,0.759472


In [40]:
#looking at the chart above, the risk when you don't have a partner is 1.22
#the greater the risk number the more important (more likely to churn)
#if risk is less than 1, the less likely
#difference and risk are very similar

In [42]:
#looking at risk ratio for gender because diff was so small for 
#both men and women, risk is closer to 1 for both.
#shows that men are as likely to churn as the
#rest of the group
#and woman are as likely to churn as the rest
df_group = df_train_full.groupby(by='gender').churn.agg(['mean'])
df_group['diff'] = df_group['mean'] - global_mean
df_group['risk'] = df_group['mean'] / global_mean
df_group

,mean,diff,risk
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


In [44]:
#since we made categorical into a list, we can do this for all cat variables
for col in categorical:
    print(col)
    df_group = df_train_full.groupby(by=col).churn.agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] - global_mean
    df_group['risk'] = df_group['mean'] / global_mean
    display(df_group)
    print()

gender


,mean,count,diff,risk
gender,,,,
female,0.276824,2796,0.006856,1.025396
male,0.263214,2838,-0.006755,0.974980



seniorcitizen


,mean,count,diff,risk
seniorcitizen,,,,
0,0.242270,4722,-0.027698,0.897403
1,0.413377,912,0.143409,1.531208



partner


...

,mean,count,diff,risk
contract,,,,
month-to-month,0.431701,3104,0.161733,1.599082
one_year,0.120573,1186,-0.149395,0.446621
two_year,0.028274,1344,-0.241694,0.104730



paperlessbilling


,mean,count,diff,risk
paperlessbilling,,,,
no,0.172071,2313,-0.097897,0.637375
yes,0.338151,3321,0.068183,1.252560



paymentmethod


,mean,count,diff,risk
paymentmethod,,,,
bank_transfer_(automatic),0.168171,1219,-0.101797,0.622928
credit_card_(automatic),0.164339,1217,-0.105630,0.608733
electronic_check,0.455890,1893,0.185922,1.688682
mailed_check,0.193870,1305,-0.076098,0.718121


# Mutual information

#### Mutual information tells us how much information we can learn about one variable if we know the value of another

In [45]:
#sci kit learn has a metrics package that does mutual information score
from sklearn.metrics import mutual_info_score

In [48]:
#to see mi in order of most important to least
def mutual_info_churn_score(series):
    return mutual_info_score(series, df_train_full.churn)

mi = df_train_full[categorical].apply(mutual_info_churn_score)
mi.sort_values(ascending=False)

contract            0.098320
onlinesecurity      0.063085
techsupport         0.061032
internetservice     0.055868
onlinebackup        0.046923
deviceprotection    0.043453
paymentmethod       0.043210
streamingtv         0.031853
streamingmovies     0.031581
paperlessbilling    0.017589
dependents          0.012346
partner             0.009968
seniorcitizen       0.009410
multiplelines       0.000857
phoneservice        0.000229
gender              0.000117
dtype: float64

In [49]:
def calculate_mi(series):
    return mutual_info_score(series, df_train_full.churn)

df_mi = df_train_full[categorical].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')


display(df_mi.head())
display(df_mi.tail())

,MI
contract,0.098320
onlinesecurity,0.063085
techsupport,0.061032
internetservice,0.055868
onlinebackup,0.046923


,MI
partner,0.009968
seniorcitizen,0.009410
multiplelines,0.000857
phoneservice,0.000229
gender,0.000117


# Correlation with numerical variables

#### a way to measure dependency between two variables
#### always remember correlation is not the same as causation

In [50]:
#here we are using the numberical list we
#created with the three variables from above
#so we want to see how much each variable correlates with churn
df_train_full[numerical].corrwith(df_train_full.churn).to_frame('correlation')

,correlation
tenure,-0.351885
monthlycharges,0.196805
totalcharges,-0.196353


In [51]:
#low correlation is 0 - 0.2, moderate is 0.2 - 0.5, high is above 0.6 - 1
#interpret this, when tenure increases (the longer they stay with us), the less they churn
# the more people pay in total, the less they leave
# the higher the monthly charges are, the more likely they will leave

In [52]:
#to look at this for tenure, do something like this
#What is the mean for people who stay with the company for 2 months?
df_train_full[df_train_full.tenure <= 2].churn.mean()

0.5953420669577875

In [54]:
#What is the mean for people who stay with the company for 12 months or more?
df_train_full[df_train_full.tenure > 12].churn.mean()

0.17634908339788277

# One-hot encoding

#### Use Scikit-Learn to encode categorical features. So you're putting both numerical and cat variables together.

In [55]:
from sklearn.feature_extraction import DictVectorizer

In [56]:
#we are putting both categorical and numerical into one dictionary
#the dictionary feature matrix can now be transformed
train_dict = df_train[categorical + numerical].to_dict(orient='records')

In [57]:
train_dict[0]

{'gender': 'female',
 'seniorcitizen': 0,
 'partner': 'yes',
 'dependents': 'yes',
 'phoneservice': 'yes',
 'multiplelines': 'yes',
 'internetservice': 'fiber_optic',
 'onlinesecurity': 'no',
 'onlinebackup': 'yes',
 'deviceprotection': 'yes',
 'techsupport': 'no',
 'streamingtv': 'yes',
 'streamingmovies': 'yes',
 'contract': 'one_year',
 'paperlessbilling': 'yes',
 'paymentmethod': 'credit_card_(automatic)',
 'tenure': 58,
 'monthlycharges': 105.2,
 'totalcharges': 6225.4}

In [58]:
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

DictVectorizer(sparse=False)

In [59]:
#transforming the train vector back into a df
X_train = dv.transform(train_dict)

In [60]:
X_train.shape

(4225, 45)

In [61]:
#doing the same for the validation dataset
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

# Logistic Regression

In [62]:
from sklearn.linear_model import LogisticRegression

In [63]:
model = LogisticRegression(solver='liblinear', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1, solver='liblinear')

In [64]:
#predict the probability
model.predict_proba(X_val)

array([[0.76799723, 0.23200277],
       [0.75226179, 0.24773821],
       [0.72286775, 0.27713225],
       ...,
       [0.997732  , 0.002268  ],
       [0.87742143, 0.12257857],
       [0.99855879, 0.00144121]])

In [65]:
y_pred = model.predict_proba(X_val)[:, 1]
y_pred

array([0.23200277, 0.24773821, 0.27713225, ..., 0.002268  , 0.12257857,
       0.00144121])

In [75]:
churn_decision = y_pred > 0.5

In [76]:
#to see the customer id's of people who will churn... send them a promotional email
#with a discount to keep them from leaving
df_val[churn_decision].customerid

10      6551-gnydg
16      0689-nkylf
18      3398-fshon
19      4704-eryfc
26      8258-gstjk
           ...    
1387    1965-ddbwu
1388    2694-ciumo
1389    9389-acwbi
1393    8837-vvwlq
1405    4273-mbhya
Name: customerid, Length: 337, dtype: object

In [77]:
(y_val == churn_decision).mean()

0.7991483321504613

In [81]:
#how many correct decisions did we make?
#compare y_val with churn_decision
df_pred = pd.DataFrame()
df_pred['probability'] = y_pred
df_pred['prediction'] = churn_decision.astype(int)
df_pred['actual'] = y_val
df_pred['correct'] = df_pred.prediction == df_pred.actual

In [84]:
df_pred

,probability,prediction,actual,correct
0,0.232003,0,0,True
1,0.247738,0,1,False
2,0.277132,0,0,True
3,0.355825,0,1,False
4,0.053981,0,0,True
...,...,...,...,...
1404,0.028544,0,0,True
1405,0.786681,1,0,False
1406,0.002268,0,0,True
1407,0.122579,0,0,True


In [85]:
#to see what percentage of time our model is correct
df_pred.correct.mean()

0.7991483321504613

In [86]:
#Our model is correct almost 80% of the time.

# Model interpretation

In [88]:
#what is the intercept of the model?
model.intercept_[0]

-0.13093793211715002

In [90]:
#use the zip function to put feature names and coefficients together

dict(zip(dv.get_feature_names(), model.coef_[0].round(3)))

{'contract=month-to-month': 0.573,
 'contract=one_year': -0.166,
 'contract=two_year': -0.538,
 'dependents=no': -0.016,
 'dependents=yes': -0.114,
 'deviceprotection=no': 0.076,
 'deviceprotection=no_internet_service': -0.114,
 'deviceprotection=yes': -0.093,
 'gender=female': -0.026,
 'gender=male': -0.105,
 'internetservice=dsl': -0.353,
 'internetservice=fiber_optic': 0.336,
 'internetservice=no': -0.114,
 'monthlycharges': 0.001,
 'multiplelines=no': -0.171,
 'multiplelines=no_phone_service': 0.112,
 'multiplelines=yes': -0.072,
 'onlinebackup=no': 0.12,
 'onlinebackup=no_internet_service': -0.114,
 'onlinebackup=yes': -0.137,
 'onlinesecurity=no': 0.263,
 'onlinesecurity=no_internet_service': -0.114,
 'onlinesecurity=yes': -0.28,
 'paperlessbilling=no': -0.207,
 'paperlessbilling=yes': 0.076,
 'partner=no': -0.107,
 'partner=yes': -0.024,
 'paymentmethod=bank_transfer_(automatic)': -0.056,
 'paymentmethod=credit_card_(automatic)': -0.116,
 'paymentmethod=electronic_check': 0.208,

In [91]:
#to make it smaller and use a subset of features contract(which has 3 columns), tenure, totalcharges
subset = ['contract', 'tenure', 'totalcharges']
train_dict_small = df_train[subset].to_dict(orient='records')
dv_small = DictVectorizer(sparse=False)
dv_small.fit(train_dict_small)

X_small_train = dv_small.transform(train_dict_small)

dv_small.get_feature_names()

['contract=month-to-month',
 'contract=one_year',
 'contract=two_year',
 'tenure',
 'totalcharges']

In [92]:
#create/train a model for the smaller subset
model_small = LogisticRegression(solver='liblinear', random_state=1)
model_small.fit(X_small_train, y_train)

LogisticRegression(random_state=1, solver='liblinear')

In [93]:
model_small.intercept_[0]

-0.6824115623552512

In [94]:
model_small.coef_[0]

array([ 9.41159158e-01, -2.05693118e-01, -1.41787760e+00, -9.41697293e-02,
        8.26215570e-04])

In [95]:
dict(zip(dv_small.get_feature_names(), model_small.coef_[0].round(3)))

{'contract=month-to-month': 0.941,
 'contract=one_year': -0.206,
 'contract=two_year': -1.418,
 'tenure': -0.094,
 'totalcharges': 0.001}

In [96]:
#interpreting this
#intercept is bias of -0.682
#weights of 5 variables is 
#monthly contract is .941
#one year contract is -0.206
#two year contract is -1.418
#tenure is -0.094
#total charges is 0.001

In [97]:
val_dict_small = df_val[subset].to_dict(orient='records')
X_small_val = dv_small.transform(val_dict_small)

In [98]:
y_pred_small = model_small.predict_proba(X_small_val)[:, 1]

# Use the model

In [100]:
#for full training dataset
dicts_train_full = df_train_full[categorical + numerical].to_dict(orient='records')

In [104]:
dv = DictVectorizer(sparse=False)
X_train_full = dv.fit_transform(dicts_train_full)
y_train_full = df_train_full.churn.values

In [105]:
#apply the log regression
model = LogisticRegression()
model.fit(X_train_full, y_train_full)

LogisticRegression()

In [106]:
#for validation dataset
dicts_test = df_test[categorical + numerical].to_dict(orient='records')

In [107]:
X_test = dv.transform(dicts_test)

In [108]:
y_pred = model.predict_proba(X_test)[:, 1]

In [109]:
#to make predictions for all clients who scored higher than 0.5
churn_decision = (y_pred >= 0.5)

In [110]:
(churn_decision == y_test).mean()

0.815471965933286

### Use this model for a specific customer

In [112]:
customer = dicts_test[10]
customer

{'gender': 'male',
 'seniorcitizen': 1,
 'partner': 'yes',
 'dependents': 'yes',
 'phoneservice': 'yes',
 'multiplelines': 'no',
 'internetservice': 'fiber_optic',
 'onlinesecurity': 'no',
 'onlinebackup': 'yes',
 'deviceprotection': 'no',
 'techsupport': 'no',
 'streamingtv': 'yes',
 'streamingmovies': 'yes',
 'contract': 'month-to-month',
 'paperlessbilling': 'yes',
 'paymentmethod': 'mailed_check',
 'tenure': 32,
 'monthlycharges': 93.95,
 'totalcharges': 2861.45}

In [113]:
X_test = dv.transform([customer])
model.predict_proba(X_test)[0, 1]

0.40568109778300626

In [115]:
#So this specific male customer will churn 40% of the time. Not likely to churn
#what was actual result? No because 0
y_test[10]

0

In [117]:
customer = dicts_test[12]
customer

{'gender': 'female',
 'seniorcitizen': 1,
 'partner': 'no',
 'dependents': 'no',
 'phoneservice': 'yes',
 'multiplelines': 'no',
 'internetservice': 'fiber_optic',
 'onlinesecurity': 'no',
 'onlinebackup': 'no',
 'deviceprotection': 'no',
 'techsupport': 'no',
 'streamingtv': 'no',
 'streamingmovies': 'no',
 'contract': 'month-to-month',
 'paperlessbilling': 'yes',
 'paymentmethod': 'electronic_check',
 'tenure': 3,
 'monthlycharges': 70.3,
 'totalcharges': 235.5}

In [118]:
X_test = dv.transform([customer])
model.predict_proba(X_test)[0, 1]

0.7587765201267906

In [119]:
#So here, this female was probably going to churn because she has probability of 75%
y_test[12]

1